## Checking photos for coords

In [7]:
import os
import pandas as pd
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
from tqdm import tqdm

# 🔧 Путь к папке с изображениями
folder_path = os.path.expanduser("~/PycharmProjects/PhotoMaps/images")
output_folder = os.path.join(os.path.dirname(folder_path), "csv")
os.makedirs(output_folder, exist_ok=True)

def convert_to_degrees(v):
    d, m, s = v
    return float(d) + float(m)/60 + float(s)/3600

def extract_gps_pillow(image_path):
    try:
        image = Image.open(image_path)
        exif_data = image._getexif()
        if not exif_data:
            return None, None

        gps_raw = {}
        for tag_id, val in exif_data.items():
            tag = TAGS.get(tag_id)
            if tag == "GPSInfo":
                for key in val:
                    gps_tag = GPSTAGS.get(key)
                    gps_raw[gps_tag] = val[key]

        if 'GPSLatitude' in gps_raw and 'GPSLongitude' in gps_raw:
            lat = convert_to_degrees(gps_raw['GPSLatitude'])
            if gps_raw.get('GPSLatitudeRef', 'N') != 'N':
                lat = -lat
            lon = convert_to_degrees(gps_raw['GPSLongitude'])
            if gps_raw.get('GPSLongitudeRef', 'E') != 'E':
                lon = -lon
            return lat, lon
        else:
            return None, None
    except Exception as e:
        print(f"❌ Ошибка при обработке {image_path}: {e}")
        return None, None

def scan_folder(path):
    records = []
    image_files = []
    for root, _, files in os.walk(path):
        for fname in files:
            if fname.lower().endswith(('.jpg', '.jpeg')):
                image_files.append(os.path.join(root, fname))

    for full_path in tqdm(image_files, desc="📍 Обработка изображений", ncols=80):
        lat, lon = extract_gps_pillow(full_path)
        records.append({
            'filename': os.path.basename(full_path),
            'folder': os.path.dirname(full_path),
            'latitude': lat,
            'longitude': lon
        })

    return pd.DataFrame(records)

# === Запуск
print(f"📁 Сканируем: {folder_path}")
df = scan_folder(folder_path)

# === Сохраняем результат
csv_path = os.path.join(output_folder, "photos.csv")
df.to_csv(csv_path, index=False)
print(f"✅ CSV сохранён: {csv_path}")

📁 Сканируем: /Users/mloktionov/PycharmProjects/PhotoMaps/images


📍 Обработка изображений: 100%|█████████████| 242/242 [00:00<00:00, 2144.82it/s]

✅ CSV сохранён: /Users/mloktionov/PycharmProjects/PhotoMaps/csv/photos.csv


In [24]:
import os
import pandas as pd
from xml.dom.minidom import Document
import re

# === НАСТРОЙКИ ===
COORDS_CSV = "photos.csv"
IDS_CSV = "drive_file_ids.csv"
KML_OUTPUT = "Poland_Map_2022_2025.kml"
BASE_URL = "https://drive.google.com/uc?export=view&id="

# === Загрузка CSV
df_coords = pd.read_csv(COORDS_CSV)
df_ids = pd.read_csv(IDS_CSV)

# === Слияние по filename
df = pd.merge(df_coords, df_ids, on='filename', how='inner')
df = df.dropna(subset=['latitude', 'longitude'])

# === Функция извлечения даты из имени файла
def extract_date(filename):
    match = re.search(r'IMG_(\d{4})(\d{2})(\d{2})', filename)
    if match:
        return f"{match.group(1)}-{match.group(2)}-{match.group(3)}"
    else:
        return filename  # если дата не найдена — оставить оригинальное имя

# === Генерация KML
doc = Document()
kml = doc.createElement('kml')
kml.setAttribute('xmlns', 'http://www.opengis.net/kml/2.2')
doc.appendChild(kml)

document = doc.createElement('Document')
kml.appendChild(document)

for _, row in df.iterrows():
    placemark = doc.createElement('Placemark')

    # Название точки = дата из имени файла
    point_name = extract_date(row['filename'])
    name = doc.createElement('name')
    name.appendChild(doc.createTextNode(point_name))
    placemark.appendChild(name)

    # Описание с <img>
    desc = doc.createElement('description')
    img_url = f"{BASE_URL}{row['file_id']}"
    html = f'<img src="{img_url}" width="400"><br>{row["filename"]}'
    cdata = doc.createCDATASection(html)
    desc.appendChild(cdata)
    placemark.appendChild(desc)

    # Координаты
    point = doc.createElement('Point')
    coord = doc.createElement('coordinates')
    coord.appendChild(doc.createTextNode(f"{row['longitude']},{row['latitude']},0"))
    point.appendChild(coord)
    placemark.appendChild(point)

    document.appendChild(placemark)

# === Сохранение файла
with open(KML_OUTPUT, "w", encoding="utf-8") as f:
    f.write(doc.toprettyxml(indent="  "))

print(f"✅ Готово! KML сохранён: {KML_OUTPUT}")

✅ Готово! KML сохранён: Poland_Map_2022_2025.kml


In [30]:
import os
from PIL import Image
from tqdm import tqdm

# === Параметры ===
SOURCE_FOLDER = "2022-2025"
OUTPUT_FOLDER = "thumbnails"
SIZE = (64, 64)

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Список всех .jpg
images = [f for f in os.listdir(SOURCE_FOLDER) if f.lower().endswith(".jpg")]

# Прогресс-бар
for fname in tqdm(images, desc="Генерация миниатюр", ncols=80):
    in_path = os.path.join(SOURCE_FOLDER, fname)
    out_path = os.path.join(OUTPUT_FOLDER, fname)
    try:
        img = Image.open(in_path)
        img.thumbnail(SIZE)
        img.save(out_path, "JPEG")
    except Exception as e:
        print(f"❌ Ошибка {fname}: {e}")

print(f"✅ Сжато {len(images)} изображений в папку '{OUTPUT_FOLDER}'")

Генерация миниатюр: 100%|█████████████████████| 242/242 [00:05<00:00, 41.39it/s]

✅ Сжато 242 изображений в папку 'thumbnails'
